<a href="https://colab.research.google.com/github/AliaksandrSiarohin/articulated-animation/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Demo for paper "Motion Representations for Articulated Animation"
**Clone repository. First install git lfs.**


In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install
!git clone https://github.com/snap-research/articulated-animation/

In [ ]:
cd articulated-animation

**Load source image and driving video.**

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('sup-mat/source.png')
driving_video = imageio.mimread('sup-mat/driving.mp4')


source_image = resize(source_image, (384, 384))[..., :3]
driving_video = [resize(frame, (384, 384))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Load checkpoints.**

In [ ]:
from demo import load_checkpoints

generator, region_predictor, avd_network = load_checkpoints(config_path='config/ted384.yaml',
                                                            checkpoint_path='checkpoints/ted384.pth')



**Perform animation.**

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, 
                             region_predictor, avd_network, animation_mode='avd')

#save resulting video
imageio.mimsave('generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())